In [1]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import torch
import os
import tqdm
import json
%matplotlib inline

In [10]:
class ObjectDetector:
    def __init__(self, model_path=None):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print(f'device = {self.device}')
        if model_path is None:
            self.model = YOLO('yolov8m.pt').to(self.device)
        else:
            self.model = YOLO(model_path).to(self.device)
            
    def predict_image(self, image_path):
        return self.model(image_path)
    
    def predict_dir(self, dir_path):
        return self.model(dir_path)
    @staticmethod
    def get_image_metadata(result, cnt, dataset_id=1):
        image_metadata = {}
        height, width = result.orig_shape
        image_metadata['id'] = cnt
        image_metadata['dataset_id'] = dataset_id
        image_metadata['path'] = result.path
        image_metadata['height'], image_metadata['width'] = height, width
        image_metadata['file_name'] = os.path.basename(result.path)
        return image_metadata

    def add_coco(self, coco_output, image_metadata, result, annotation_id):
        coco_output["images"].append({
            "id": image_metadata["id"],
            "dataset_id": image_metadata["dataset_id"],
            "category_ids": [],
            "path": os.path.join('datasets', image_metadata["path"]),
            "width": image_metadata["width"],
            "height": image_metadata["height"],
            "file_name": image_metadata["file_name"],
            "annotated": False,
            "annotating": [],
            "num_annotations": 0,
            "metadata": {},
            "deleted": False,
            "milliseconds": 0,
            "events": [],
            "regenerate_thumbnail": False
        })
        for x in result.boxes:
            xmin, ymin, xmax, ymax = x.cpu().numpy().xyxy[0]
            xmin, ymin, xmax, ymax = xmin.item(), ymin.item(), xmax.item(), ymax.item()
            ymin, xmin, ymax, xmax = round(ymin, 1), round(xmin, 1), round(ymax, 1), round(xmax, 1)
            x, y, w, h = xmin, ymin, (xmax - xmin), (ymax - ymin)
            x, y, w, h = round(x, 1), round(y, 1), round(w, 1), round(h, 1)
            segmentation_points = [xmin, ymin, xmax, ymin, xmax, ymax, xmin, ymax]
            coco_output["annotations"].append({
                "id": annotation_id,
                "image_id": image_metadata["id"],
                "category_id": 1,
                "segmentation": [segmentation_points], 
                "area": round(w * h, 0),
                "bbox": [x, y, w, h],
                "iscrowd": False,
                "isbbox": True,
                "color": "#8eb517", 
                "metadata": {}
            })
            annotation_id += 1
        return annotation_id
        
        
    def coco_annotate(self, dir_path, output_path='labels.json'):
        results = self.predict_dir(dir_path)
        coco_output = {
            "images": [],
            "categories": [
                {"id": 1, "name": "Elephant", "supercategory": "", "color": "#3ab7dd", "metadata": {}, "keypoint_colors": []}
            ],
            "annotations": []
        }
        annotation_id = 1
        for i, result in enumerate(results, 1):
            #print(result)
            image_metadata = self.get_image_metadata(result, i)
            annotation_id = self.add_coco(coco_output, image_metadata, result, annotation_id)
        print(coco_output)
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(coco_output, f, ensure_ascii=False, indent=4)
        
    def show(self, img):
        plt.figure()
        plt.axis('off')
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)) 
        plt.show()
    def show_labeled(self, img_path):
        result = self.predict_image(img_path)
        labeled_img = result[0].plot()
        self.show(labeled_img)

        
        
object_detector = ObjectDetector('best.pt')
object_detector.coco_annotate('cx')

device = cuda

image 1/12 /home/improcloud/nguyehu7/BP/cx/screenshot2_09_02__20_45.png: 352x640 1 Elephant, 11.9ms
image 2/12 /home/improcloud/nguyehu7/BP/cx/screenshot2_09_02__21_00.png: 352x640 (no detections), 11.0ms
image 3/12 /home/improcloud/nguyehu7/BP/cx/screenshot2_09_02__22_15.png: 352x640 2 Elephants, 11.0ms
image 4/12 /home/improcloud/nguyehu7/BP/cx/screenshot2_09_02__22_30.png: 352x640 3 Elephants, 11.0ms
image 5/12 /home/improcloud/nguyehu7/BP/cx/screenshot2_09_02__23_30.png: 352x640 3 Elephants, 11.1ms
image 6/12 /home/improcloud/nguyehu7/BP/cx/screenshot2_10_02__15_30.png: 352x640 3 Elephants, 11.1ms
image 7/12 /home/improcloud/nguyehu7/BP/cx/screenshot2_10_02__16_00.png: 352x640 3 Elephants, 11.0ms
image 8/12 /home/improcloud/nguyehu7/BP/cx/screenshot2_10_02__18_30.png: 352x640 1 Elephant, 11.0ms
image 9/12 /home/improcloud/nguyehu7/BP/cx/screenshot2_11_02__00_45.png: 352x640 3 Elephants, 11.0ms
image 10/12 /home/improcloud/nguyehu7/BP/cx/screenshot2_11_02__16_00.png: 